In [1]:
# @inproceedings{karkkainenfairface,
#   title={FairFace: Face Attribute Dataset for Balanced Race, Gender, and Age for Bias Measurement and Mitigation},
#   author={Karkkainen, Kimmo and Joo, Jungseock},
#   booktitle={Proceedings of the IEEE/CVF Winter Conference on Applications of Computer Vision},
#   year={2021},
#   pages={1548--1558}
# }

In [4]:
import pandas as pd
import dlib
from tqdm import tqdm

## Steps:
1. Combine training and validation dataset
    - There's no visual difference between them so might as well combine
2. Filter them according to a face recognition algo
    - HAAR cascade
    - Only keep images that have a detectable face
3. Split dataset into 70/20/10 (training/validation/testing)
4. Perturb 10% of the dataset:
    - Makeup
    - Glasses
    - Mask

In [2]:
fairface_train = pd.read_csv("fairface_label_train.csv")
fairface_val = pd.read_csv("fairface_label_val.csv")

In [58]:
fairface_val.groupby("gender").nunique()

,file,age,race,service_test
gender,,,,
Female,5162,9,7,2
Male,5792,9,7,2


In [45]:
def give_filenames(dataset):
    male_state = dataset['gender'] == 'Male'
    female_state = dataset['gender'] == 'Female'
    male = dataset[male_state]
    female = dataset[female_state]
    
    male_filenames = []
    for i in tqdm(range(len(male)), "Loading male files..."):
        male_filenames.append(male.iloc[i]['file'])
        
    female_filenames = []
    for i in tqdm(range(len(female)), "Loading female files..."):
        female_filenames.append(female.iloc[i]['file'])
    return male_filenames, female_filenames

In [30]:
male_filenames, female_filenames = give_filenames(fairface_train)

Loading female files...: 100%|██████████| 40758/40758 [00:04<00:00, 9264.34it/s]


## For Training Dataset

In [46]:
# Split training dataset to male and female

import os
import shutil
for i in tqdm(range(len(male_filenames)), desc="Loading files..."):
    file = male_filenames[i]
    filename_from = os.path.join('fairface-img-margin025-trainval/', file)
    filename_to = os.path.join('male/', file[6:])  #     os.rename(filename_from, filename_to)
    try:
        if (os.path.exists(filename_from)):
            shutil.move(filename_from, filename_to)
            os.replace(filename_from, filename_to)
            filein.close()
    except:
        continue

Loading files...: 100%|██████████| 45986/45986 [00:00<00:00, 52379.94it/s]


## For Testing Dataset

In [47]:
male_fn, female_fn = give_filenames(fairface_val)

Loading female files...: 100%|██████████| 5162/5162 [00:00<00:00, 9247.03it/s]


In [53]:
# Split val dataset to male and female

import os
import shutil
for i in tqdm(range(len(female_fn)), desc="Loading files..."):
    file = female_fn[i]
    filename_from = os.path.join('fairface-img-margin025-trainval/', file)
    filename_to = os.path.join('female/', file[4:])
#     os.rename(filename_from, filename_to)
    try:
        if (os.path.exists(filename_from)):
            shutil.move(filename_from, filename_to)
            os.replace(filename_from, filename_to)
            filein.close()
    except:
        continue

Loading files...: 100%|██████████| 5162/5162 [00:00<00:00, 54718.73it/s]


## Make Up Code

In [56]:
from pathlib import Path

In [142]:
from PIL import Image, ImageDraw
import face_recognition

def appl_makeup(filename, output_fn):
    """
    Applies make up filter on a single image and saves it to a given directory.
    
    filename : str
        Input image
    output_fn : str
        Output image to save
    """
    # Load the jpg file into a numpy array
    image = face_recognition.load_image_file(filename)

    # Find all facial features in all the faces in the image
    face_landmarks_list = face_recognition.face_landmarks(image)

    pil_image = Image.fromarray(image)
    for face_landmarks in face_landmarks_list:
        d = ImageDraw.Draw(pil_image, 'RGBA')

        # Make the eyebrows into a nightmare
        d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
        d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
        d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
        d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

        # Gloss the lips
        d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
        d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
        d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
        d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

        # Sparkle the eyes
        d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
        d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

        # Apply some eyeliner
        d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
        d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)
        fn = Path(output_fn)
        if not fn.is_dir():
            pil_image.save(output_fn)    # Change this to male or female

## Make Up for Male Training Dataset

In [46]:
male_fn, female_fn = give_filenames(fairface_train)

NameError: name 'fairface_train' is not defined

## Glasses

In [1]:
from insta_filters.put_glasses import run_filter
import shutil
import numpy as np
import cv2

In [4]:
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [2]:
filename='1513.jpg'
img=cv2.imread(filename)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    cv2.imwrite("test.png", img)

In [3]:
eye_cascade.detectMultiScale(roi_gray)

NameError: name 'roi_gray' is not defined